# Development

In [ ]:
%load_ext autoreload
%autoreload 2
import nima_io
import nima_io.read as ir

# import skimage.io
# import matplotlib
# import matplotlib.pyplot as plt
# %matplotlib inline
# nd2 = "/home/dan/pims/pims/tests/data/bioformats/cluster.nd2"
import numpy as np
from pathlib import Path

tdata = Path("../../tests/data/")

lif = tdata / "2015Aug28_TransHXB2_50min+DMSO.lif"
img_tile = tdata / "t4_1.tif"  # C=3 T=4 S=15
img_void_tile = tdata / "tile6_1.tif"  # C=4 T=3 S=14 scattered
# imgsingle = tdata / "exp2_2.tif"  # C=2 T=81

# mcts = tdata / "multi-channel-time-series.ome.tif"  # C=3 T=7
# bigtiff = tdata / "LC26GFP_1.tf8"  # bigtiff

slif = str(lif)
simg_tile = str(img_tile)
simg_void_tile = str(img_void_tile)
# simgsingle = str(imgsingle)
# smcts = str(mcts)Z`Z
# sbigtiff = str(bigtiff)

Core metadata list at:
https://docs.openmicroscopy.org/bio-formats/7.1.0/developers/file-reader.html



In [ ]:
md, wr = ir.read(slif)

In [ ]:
wr.rdr.setSeries(4)
wr.rdr.getImageCount()

In [ ]:
md

In [ ]:
md, wr = ir.read(slif)

In [ ]:
wr.rdr.setSeries(4)
wr.rdr.getImageCount()

In [ ]:
md_store = wr.rdr.getMetadataStore()

In [ ]:
md_store.getPlaneCount(4), md_store.getPlaneTheC(4, 110), md_store.getPixelsSizeZ(4)

In [ ]:
wr.rdr.getDimensionOrder()

In [ ]:
v = md_store.getPixelsPhysicalSizeY(0)
v.value(), v.unit, ir.convert_value(v)

## read2 

In [ ]:
md, wr = ir.read(simg_tile)
md2, mdd2 = ir.read2(simg_tile)
md_store = wr.rdr.getMetadataStore()

In [ ]:
md2.keys(), md2["FolderCount"]

In [ ]:
md2["PlanePositionX"]

In [ ]:
extra_md, mdd = ir.get_md_dict(md_store, simg_tile)

In [ ]:
extra_md["PlanePositionX"]

In [ ]:
ir.get_allvalues_grouped(md_store, "getPixelsBigEndian", 1, True)

In [ ]:
md, mdd = ir.get_md_dict(md_store)
len(md)

In [ ]:
mdd

In [ ]:
root = md_store.getRoot()
im0 = root.getImage(0)
pixels = im0.getPixels()

In [ ]:
import io

plane = pixels.getPlane(3)

output = io.StringIO()
output.write(str(plane.getDeltaT()))
contents = output.getvalue()
output.close()

In [ ]:
output = io.StringIO()
output.write(str(plane.getDeltaT()))
contents = output.getvalue()
output.close()
contents

In [ ]:
md, missing_keys = ir.get_md_dict(md_store)

In [ ]:
md["PixelsSizeZ"]

In [ ]:
getattr(md_store, "getChannelExcitationWavelength")(0, 1)

## XML
getRoot was used in imgread-0.2.1

In [ ]:
instrument = root.getInstrument(0)
detector = instrument.getDetector(0)
detector.getModel()

In [ ]:
wr.rdr.getMetadataStoreRoot().getInstrument(0).getDetector(0).getModel()

In [ ]:
wr.rdr.getMetadataStore().getRoot().getInstrument(0).getDetector(0).getModel()

In [ ]:
[m for m in root.__dir__() if m[:3] == "get"]

In [ ]:
image = root.getImage(13)
[m for m in image.__dir__() if m[:3] == "get"]

In [ ]:
obj = image.getObjectiveSettings()
[m for m in obj.__dir__() if m[:3] == "get"]
obj.getID()

In [ ]:
ch = pixels.getChannel(0)
[m for m in ch.__dir__() if m[:3] == "get"]

In [ ]:
lp = ch.getLightPath()
[m for m in lp.__dir__() if m[:3] == "get"]

In [ ]:
pixels = image.getPixels()
[m for m in pixels.__dir__() if m[:3] == "get"]

In [ ]:
pixels.getTimeIncrement()

In [ ]:
plane = pixels.getPlane(11)
[m for m in plane.__dir__() if m[:3] == "get"]

In [ ]:
import jpype

jpype.JObject(plane.getTheC()), plane.getDeltaT(), plane.getExposureTime()

In [ ]:
for k in root.__dir__():
    if k[:3] == "get":
        print(k)

In [ ]:
wr.rdr.getMetadataStore().getExperimentID(0)

In [ ]:
root.getExperiment(0)

In [ ]:
im = root.getImage(0)
obj = im.getObjectiveSettings()
obj.getID(), obj.getMedium(), obj.getRefractiveIndex(), obj.getCorrectionCollar(), obj.getObjective()

In [ ]:
wr.rdr.getMetadataStore().getObjectiveID(
    0, 0
) == obj.getID(), wr.rdr.getMetadataStore().getObjectiveLensNA(0, 0)

## md vs root (to remember)

from md i can get info on camera, objectives ...

but also key parameters like exposuretime, DeltaT, planePositionXYZ

In [ ]:
wr.rdr.getMetadataStore().getObjectiveCount(0)

In [ ]:
metadata = wr.rdr.getMetadataStore()
# might be used to check if metadata are the same for all series
(
    metadata.getObjectiveCount(0),
    metadata.getObjectiveImmersion(0, 0),
    metadata.getPixelsDimensionOrder(0),
    metadata.getPixelsPhysicalSizeX(0),
    # metadata.getTiffDataIFD(0, 5), only fei
    metadata.getObjectiveLensNA(0, 0),
    metadata.getChannelCount(0),
    metadata.getDetectorSettingsBinning(0, 1),  # series, channel
    metadata.getDetectorModel(0, 0),
    metadata.getDetectorType(0, 0),
    metadata.getTiffDataCount(0),
    metadata.getPlaneExposureTime(0, 1),  # none.value() catch this error with try
    metadata.getPlaneDeltaT(0, 2).value(),
    metadata.getPlaneDeltaT(0, 11).value().doubleValue(),
    metadata.getPlanePositionX(0, 0),
    metadata.getScreenCount(),
    # metadata.getXMLAnnotationCount(),
)
# cerca tutti i ...Count()

In [ ]:
metadata.getXMLAnnotationValue(
    0
) == metadata.getRoot().getStructuredAnnotations().getXMLAnnotation(0).getValue()

In [ ]:
rr = wr.rdr.getMetadataStore().getRoot()

In [ ]:
rr.getStructuredAnnotations().getXMLAnnotation(0).getValue()

In [ ]:
# import bioformats

# md = bioformats.get_omexml_metadata(slif)
from xml.etree import ElementTree as ETree

# mdroot = ETree.fromstring(wr.rdr.getMetadataStore().dumpXML())
java_string = (
    wr.rdr.getMetadataStoreRoot()
    .getStructuredAnnotations()
    .getXMLAnnotation(0)
    .getValue()
)
python_string = jpype.java.lang.String(java_string).toString()
mdroot = ETree.fromstring(python_string.getBytes())

for a in mdroot:
    print((a.tag, a.attrib))
    for aa in a:
        print((" GGSSS ", aa.tag, aa.attrib))
        for aaa in aa:
            print((" GGSSS2 ", aaa.tag, aaa.attrib))

In [ ]:
import xmltodict

m1 = xmltodict.parse(metadata.dumpXML().getBytes())
m1["OME"]

In [ ]:
metalist = [
    (child.tag, [[child.attrib]])
    for child in ETree.fromstring(metadata.dumpXML().getBytes()).iter()
]

In [ ]:
m = metalist[10]
m[0], m[1][0][0]

In [ ]:
for child in mdroot:
    if child.tag.endswith(""):
        for grandchild in child:
            print((grandchild.tag, grandchild.attrib))

## Jpype

In [ ]:
rj = ir.read_jpype(slif)

In [ ]:
rj[0]

In [ ]:
filepath = simg_void_tile
loci = jpype.JPackage("loci")
rdr = loci.formats.ImageReader()
rdr.setMetadataStore(loci.formats.MetadataTools.createOMEXMLMetadata())
rdr.setId(filepath)
xml_md = rdr.getMetadataStore()

In [ ]:
md, mdd = ir.get_md_dict(xml_md, debug=True)
md

In [ ]:
print(md["ImageCount"][0][1], md["PlaneCount"])
print(md["PlanePositionX"][3][1][0])
print(md["ImageAcquisitionDate"])
print(md["PixelsPhysicalSizeX"][0][1][0])
print(md["PixelsType"][0][1])

print(md["PlanePositionZ"])

In [ ]:
md0, rdrdt = ir.read_jpype(simg_tile)
rdrdt, md0

In [ ]:
mmd = rdrdt[2]

In [ ]:
mmd["ArcType"]

In [ ]:
rdr = rdrdt[0]

In [ ]:
rdr.getPixelType(), rdr.getDimensionOrder(),

In [ ]:
r = rdr.getMetadataStoreRoot()

In [ ]:
reader = rdr.getReader()

In [ ]:
r.getImage

In [ ]:
rdr.getIndex(0, 3, 2)

In [ ]:
import jpype

loci = jpype.JPackage("loci")

cs = loci.formats.ChannelSeparator(reader)
fs = loci.formats.Memoizer(reader)